# Examples of how to build concept-based explanations

Load the model and list modules to find where to split it.

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("gpt2")

print(list(model.named_children()))
del model

### Split the model using the `ModelWithSplitPoints` class

In [ ]:
from interpreto import ModelWithSplitPoints

splitted_model = ModelWithSplitPoints(
    model_or_repo_id="gpt2",
    split_points="transformer.h.1.mlp",
    model_autoclass=AutoModelForCausalLM,
)

### Load the dataset and compute activations

In [ ]:
from datasets import load_dataset

rotten_tomatoes = load_dataset("cornell-movie-review-data/rotten_tomatoes")["train"]["text"]

activations = splitted_model.get_activations(rotten_tomatoes[100:])

activations["transformer.h.1.mlp"].shape

### Create and fit the concept explainer

In [ ]:
from interpreto.concepts import ICAConcepts

concept_explainer = ICAConcepts(splitted_model, nb_concepts=10)

concept_explainer.fit(activations)

### Interpret the concepts

In [ ]:
import os

from interpreto.concepts.interpretations.llm_labels import SAMPLING_METHOD, LLMLabels
from interpreto.model_wrapping.llm_interface import OpenAILLM
from interpreto.model_wrapping.model_with_split_points import ActivationGranularity

interpretations = concept_explainer.interpret(
    LLMLabels,
    concepts_indices=[0, 1, 2],
    activation_granularity=ActivationGranularity.SAMPLE,
    llm_interface=OpenAILLM(api_key=os.environ["OPENAI_API_KEY"]),
    sampling_method=SAMPLING_METHOD.TOP,
    inputs=rotten_tomatoes[:20],
    k_context=3,
)

In [ ]:
interpretations